<a href="https://colab.research.google.com/github/Nidhig19/NLP/blob/main/notebooks/TestCaseNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [5]:
!pip install spacy gensim
!pip install tabulate

In [6]:
!git clone https://github.com/Nidhig19/NLP.git

fatal: destination path 'NLP' already exists and is not an empty directory.


In [7]:
!python -m spacy download en_core_web_md

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.7.1/en_core_web_md-3.7.1-py3-none-any.whl (42.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import spacy, gensim
from tabulate import tabulate
from spacy import displacy
nlp = spacy.load('en_core_web_md')

# Data Preprocessing

In [10]:
#Import and read file
with open('/content/NLP/data/7zbk8zsd8y-1/g02-federalspending.txt') as file:
    sample = file.read()
text = nlp(sample)
sentence_spans = list(text.sents)

In [11]:
for sentence in sentence_spans:
    print(sentence)

﻿As a Data user, I want to have the 12-19-2017 deletions processed.

As a UI designer, I want to redesign the Resources page, so that it matches the new Broker design styles.

As a UI designer, I want to report to the Agencies about user testing, so that they are aware of their contributions to making Broker a better UX.

As a UI designer, I want to move on to round 2 of DABS or FABS landing page edits, so that I can get approvals from leadership.

As a UI designer, I want to move on to round 2 of Homepage edits, so that I can get approvals from leadership.

As a UI designer, I want to move on to round 3 of the Help page edits, so that I can get approvals from leadership.

As a Developer , I want to be able to log better, so that I can troubleshoot issues with particular submissions and functions.

As a Developer, I want to add the updates on a FABS submission to be modified when the publishStatus changes, so that I know when the status of the submission has changed.

As a DevOps engin

In [12]:
#Removing punctuations, stop words, whitespaces
sentence_tokens=[]
for i,sentence in enumerate(sentence_spans):
    filtered_text=[token for token in sentence if not token.is_punct and not token.is_stop and not token.is_space]
    sentence_tokens.append(filtered_text)
    print(i+1,filtered_text)

1 [﻿As, Data, user, want, 12, 19, 2017, deletions, processed]
2 [UI, designer, want, redesign, Resources, page, matches, new, Broker, design, styles]
3 [UI, designer, want, report, Agencies, user, testing, aware, contributions, making, Broker, better, UX]
4 [UI, designer, want, round, 2, DABS, FABS, landing, page, edits, approvals, leadership]
5 [UI, designer, want, round, 2, Homepage, edits, approvals, leadership]
6 [UI, designer, want, round, 3, Help, page, edits, approvals, leadership]
7 [Developer, want, able, log, better, troubleshoot, issues, particular, submissions, functions]
8 [Developer, want, add, updates, FABS, submission, modified, publishStatus, changes, know, status, submission, changed]
9 [DevOps, engineer, want, New, Relic, provide, useful, data, applications]
10 [UI, designer, want, round, 2, Help, page, edits, approvals, leadership]
11 [UI, designer, want, round, 2, Homepage, edits, approvals, leadership]
12 [Broker, user, want, Upload, Validate, error, message, accu

# Input, Action and Condition Words


In [13]:
from spacy.tokens import Doc

In [14]:
def custom_dep_tree(span, main_verb_index):
  words = [token.text for token in span]
  new_doc = Doc(span.vocab, words=words)

  for i, token in enumerate(span):
    new_doc[i].pos_ = token.pos_
    new_doc[i].lemma_ = token.lemma_
    new_doc[i].tag_ = token.tag_
    new_doc[i].dep_ = token.dep_
    head_index = min(token.head.i - span[0].i, len(new_doc) - 1)
    new_doc[i].head = new_doc[head_index]

  if main_verb_index is not None:
    # Set the main verb as root
    new_doc[main_verb_index].dep_ = "ROOT"
    new_doc[main_verb_index].head = new_doc[main_verb_index]

  # Adjust dependencies for other tokens
  for token in new_doc:
      if token.i != main_verb_index and token.dep_ == "ROOT":
          token.dep_ = "dep"
          token.head = new_doc[main_verb_index]

  return new_doc


In [15]:
res = []
for id,tokens in enumerate(sentence_tokens):
  main_verb_index = None
  results = {}
  for i, token in enumerate(tokens):

    if token.lemma_ == "want":
      for j, next_token in enumerate(tokens[i:], start=i):
        if (next_token.pos_ == "VERB" and
          next_token.lemma_ not in ["want", "be", "have"] and
          not next_token.dep_ == "aux"):
          main_verb_index = j
          break

      if main_verb_index:
        break

  if main_verb_index is not None:

    main_verb_text = tokens[main_verb_index].text
    orig_main_verb_index = next(
        (i for i, token in enumerate(sentence_spans[id])
         if token.text == main_verb_text),
        None
    )

    modified_doc = custom_dep_tree(sentence_spans[id], orig_main_verb_index)
    main_verb = modified_doc[orig_main_verb_index]
    # print(main_verb)


    # Role Extraction
    role = None
    for i, token in enumerate(sentence_spans[id]):
        if token.text.lower() == "as" and i + 2 < len(sentence_spans[id]):
            # Collect role words
            role_words = []
            for j in range(i + 2, len(sentence_spans[id])):
                current_token = sentence_spans[id][j]
                if current_token.text.lower() == "," and j + 1 < len(sentence_spans[id]) and sentence_spans[id][j+1].text.lower() == "i":
                    break
                role_words.append(current_token.text)

            role = " ".join(role_words)
            break


    # Condition Extraction
    condns = None
    for i, token in enumerate(sentence_spans[id]):
        if (token.text.lower() == "so"):
            # Collect condition words
            condn_words = []
            for j in range(i, len(sentence_spans[id])):
                current_token = sentence_spans[id][j]
                if current_token.text.lower() == ".":
                  break
                condn_words.append(current_token.text)

            condns = " ".join(condn_words)
            break


  results = {
          'main_action': main_verb.text,
          'role': role or "Unknown",
          'conditions': condns or "Unknown",
          'original_text': sentence_spans[id],
          'filtered_text':tokens,
          'main_verb_index': main_verb_index,
          'dependencies': [(token.text, token.dep_, token.head.text)
                          for token in modified_doc if token.i != orig_main_verb_index],
          'modified_doc': modified_doc
      }
  res.append(results)
  print("\n", results['original_text'])
  print("Main Action:", results['main_action'])
  print("Role:", results['role'])
  print("Conditions:", results['conditions'])
  print("Dependencies: ", results['dependencies'])


 ﻿As a Data user, I want to have the 12-19-2017 deletions processed.

Main Action: processed
Role: Unknown
Conditions: Unknown
Dependencies:  [('\ufeffAs', 'nummod', 'user'), ('a', 'det', 'user'), ('Data', 'compound', 'user'), ('user', 'dep', 'want'), (',', 'punct', 'want'), ('I', 'nsubj', 'want'), ('want', 'dep', 'processed'), ('to', 'aux', 'have'), ('have', 'xcomp', 'want'), ('the', 'det', 'deletions'), ('12', 'nummod', 'deletions'), ('-', 'punct', '2017'), ('19', 'nummod', '2017'), ('-', 'punct', '2017'), ('2017', 'prep', '12'), ('deletions', 'dobj', 'have'), ('.', 'punct', 'want'), ('\n', 'dep', '.')]

 As a UI designer, I want to redesign the Resources page, so that it matches the new Broker design styles.

Main Action: redesign
Role: UI designer
Conditions: so that it matches the new Broker design styles
Dependencies:  [('As', 'prep', 'want'), ('a', 'det', 'designer'), ('UI', 'compound', 'designer'), ('designer', 'pobj', 'As'), (',', 'punct', 'want'), ('I', 'nsubj', 'want'), ('w

# Dependency Parsing

In [16]:
for i,result in enumerate(res):
    data = []
    displacy.render(result['modified_doc'], style="dep")
    filtered_words = [token.text for token in result['filtered_text']]
    filtered_tokens = [token for token in result['modified_doc'] if token.text in filtered_words]
    for token in filtered_tokens:
        data.append([token.text, token.lemma_, token.pos_, token.tag_, token.dep_])

    # Print the table
    headers = ["Text", "Lemma", "POS", "Tag", "Dependency"]
    print(tabulate(data, headers=headers, tablefmt="fancy_grid"))

╒═══════════╤══════════╤═══════╤═══════╤══════════════╕
│ Text      │ Lemma    │ POS   │ Tag   │ Dependency   │
╞═══════════╪══════════╪═══════╪═══════╪══════════════╡
│ ﻿As        │ ﻿as       │ NUM   │ CD    │ nummod       │
├───────────┼──────────┼───────┼───────┼──────────────┤
│ Data      │ Data     │ PROPN │ NNP   │ compound     │
├───────────┼──────────┼───────┼───────┼──────────────┤
│ user      │ user     │ NOUN  │ NN    │ dep          │
├───────────┼──────────┼───────┼───────┼──────────────┤
│ want      │ want     │ VERB  │ VBP   │ dep          │
├───────────┼──────────┼───────┼───────┼──────────────┤
│ 12        │ 12       │ NUM   │ CD    │ nummod       │
├───────────┼──────────┼───────┼───────┼──────────────┤
│ 19        │ 19       │ NUM   │ CD    │ nummod       │
├───────────┼──────────┼───────┼───────┼──────────────┤
│ 2017      │ 2017     │ NUM   │ CD    │ prep         │
├───────────┼──────────┼───────┼───────┼──────────────┤
│ deletions │ deletion │ NOUN  │ NNS   │ dobj 

╒═══════════╤═══════════╤═══════╤═══════╤══════════════╕
│ Text      │ Lemma     │ POS   │ Tag   │ Dependency   │
╞═══════════╪═══════════╪═══════╪═══════╪══════════════╡
│ UI        │ ui        │ NOUN  │ NN    │ compound     │
├───────────┼───────────┼───────┼───────┼──────────────┤
│ designer  │ designer  │ NOUN  │ NN    │ pobj         │
├───────────┼───────────┼───────┼───────┼──────────────┤
│ want      │ want      │ VERB  │ VBP   │ dep          │
├───────────┼───────────┼───────┼───────┼──────────────┤
│ redesign  │ redesign  │ VERB  │ VB    │ ROOT         │
├───────────┼───────────┼───────┼───────┼──────────────┤
│ Resources │ Resources │ PROPN │ NNPS  │ compound     │
├───────────┼───────────┼───────┼───────┼──────────────┤
│ page      │ page      │ NOUN  │ NN    │ dobj         │
├───────────┼───────────┼───────┼───────┼──────────────┤
│ matches   │ match     │ VERB  │ VBZ   │ advcl        │
├───────────┼───────────┼───────┼───────┼──────────────┤
│ new       │ new       │ ADJ  

╒═══════════════╤══════════════╤═══════╤═══════╤══════════════╕
│ Text          │ Lemma        │ POS   │ Tag   │ Dependency   │
╞═══════════════╪══════════════╪═══════╪═══════╪══════════════╡
│ UI            │ ui           │ NOUN  │ NN    │ compound     │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ designer      │ designer     │ NOUN  │ NN    │ pobj         │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ want          │ want         │ VERB  │ VBP   │ dep          │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ report        │ report       │ VERB  │ VB    │ ROOT         │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ Agencies      │ agency       │ NOUN  │ NNS   │ pobj         │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ user          │ user         │ NOUN  │ NN    │ compound     │
├───────────────┼──────────────┼───────┼───────┼──────────────┤
│ testing       │ testing      │ NOUN  │

╒════════════╤════════════╤═══════╤═══════╤══════════════╕
│ Text       │ Lemma      │ POS   │ Tag   │ Dependency   │
╞════════════╪════════════╪═══════╪═══════╪══════════════╡
│ UI         │ ui         │ NOUN  │ NN    │ compound     │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ designer   │ designer   │ NOUN  │ NN    │ pobj         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ want       │ want       │ VERB  │ VBP   │ dep          │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ round      │ round      │ VERB  │ VB    │ ROOT         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ 2          │ 2          │ NUM   │ CD    │ dobj         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ DABS       │ DABS       │ PROPN │ NNP   │ nmod         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ FABS       │ FABS       │ PROPN │ NNP   │ compound     │
├────────────┼────────────┼───────┼───────┼─────────────

╒════════════╤════════════╤═══════╤═══════╤══════════════╕
│ Text       │ Lemma      │ POS   │ Tag   │ Dependency   │
╞════════════╪════════════╪═══════╪═══════╪══════════════╡
│ UI         │ ui         │ NOUN  │ NN    │ compound     │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ designer   │ designer   │ NOUN  │ NN    │ pobj         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ want       │ want       │ VERB  │ VBP   │ dep          │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ round      │ round      │ VERB  │ VB    │ ROOT         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ 2          │ 2          │ NUM   │ CD    │ dobj         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ Homepage   │ homepage   │ NOUN  │ NN    │ compound     │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ edits      │ edit       │ NOUN  │ NNS   │ pobj         │
├────────────┼────────────┼───────┼───────┼─────────────

...

╒═════════════╤════════════╤═══════╤═══════╤══════════════╕
│ Text        │ Lemma      │ POS   │ Tag   │ Dependency   │
╞═════════════╪════════════╪═══════╪═══════╪══════════════╡
│ FABS        │ fabs       │ NOUN  │ NN    │ compound     │
├─────────────┼────────────┼───────┼───────┼──────────────┤
│ user        │ user       │ NOUN  │ NN    │ pobj         │
├─────────────┼────────────┼───────┼───────┼──────────────┤
│ want        │ want       │ VERB  │ VBP   │ dep          │
├─────────────┼────────────┼───────┼───────┼──────────────┤
│ validations │ validation │ NOUN  │ NNS   │ nsubj        │
├─────────────┼────────────┼───────┼───────┼──────────────┤
│ run         │ run        │ VERB  │ VBN   │ ROOT         │
├─────────────┼────────────┼───────┼───────┼──────────────┤
│ reasonable  │ reasonable │ ADJ   │ JJ    │ amod         │
├─────────────┼────────────┼───────┼───────┼──────────────┤
│ time        │ time       │ NOUN  │ NN    │ pobj         │
╘═════════════╧════════════╧═══════╧════

╒════════╤═════════╤═══════╤═══════╤══════════════╕
│ Text   │ Lemma   │ POS   │ Tag   │ Dependency   │
╞════════╪═════════╪═══════╪═══════╪══════════════╡
│ FABS   │ fabs    │ NOUN  │ NN    │ compound     │
├────────┼─────────┼───────┼───────┼──────────────┤
│ user   │ user    │ NOUN  │ NN    │ pobj         │
├────────┼─────────┼───────┼───────┼──────────────┤
│ want   │ want    │ VERB  │ VBP   │ dep          │
╘════════╧═════════╧═══════╧═══════╧══════════════╛


╒════════════╤════════════╤═══════╤═══════╤══════════════╕
│ Text       │ Lemma      │ POS   │ Tag   │ Dependency   │
╞════════════╪════════════╪═══════╪═══════╪══════════════╡
│ agency     │ agency     │ NOUN  │ NN    │ compound     │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ user       │ user       │ NOUN  │ NN    │ pobj         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ want       │ want       │ VERB  │ VBP   │ dep          │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ know       │ know       │ VERB  │ VB    │ ROOT         │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ submission │ submission │ NOUN  │ NN    │ compound     │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ periods    │ period     │ NOUN  │ NNS   │ nsubj        │
├────────────┼────────────┼───────┼───────┼──────────────┤
│ start      │ start      │ VERB  │ VBP   │ ccomp        │
├────────────┼────────────┼───────┼───────┼─────────────

╒══════════╤══════════╤═══════╤═══════╤══════════════╕
│ Text     │ Lemma    │ POS   │ Tag   │ Dependency   │
╞══════════╪══════════╪═══════╪═══════╪══════════════╡
│ agency   │ agency   │ NOUN  │ NN    │ compound     │
├──────────┼──────────┼───────┼───────┼──────────────┤
│ user     │ user     │ NOUN  │ NN    │ pobj         │
├──────────┼──────────┼───────┼───────┼──────────────┤
│ want     │ want     │ VERB  │ VBP   │ dep          │
├──────────┼──────────┼───────┼───────┼──────────────┤
│ landing  │ landing  │ NOUN  │ NN    │ compound     │
├──────────┼──────────┼───────┼───────┼──────────────┤
│ page     │ page     │ NOUN  │ NN    │ nsubj        │
├──────────┼──────────┼───────┼───────┼──────────────┤
│ navigate │ navigate │ VERB  │ VB    │ ROOT         │
├──────────┼──────────┼───────┼───────┼──────────────┤
│ FABS     │ FABS     │ PROPN │ NNP   │ nmod         │
├──────────┼──────────┼───────┼───────┼──────────────┤
│ DABS     │ DABS     │ PROPN │ NNP   │ conj         │
├─────────

╒════════════╤═══════════╤═══════╤═══════╤══════════════╕
│ Text       │ Lemma     │ POS   │ Tag   │ Dependency   │
╞════════════╪═══════════╪═══════╪═══════╪══════════════╡
│ agency     │ agency    │ NOUN  │ NN    │ compound     │
├────────────┼───────────┼───────┼───────┼──────────────┤
│ user       │ user      │ NOUN  │ NN    │ pobj         │
├────────────┼───────────┼───────┼───────┼──────────────┤
│ want       │ want      │ VERB  │ VBP   │ dep          │
├────────────┼───────────┼───────┼───────┼──────────────┤
│ submit     │ submit    │ VERB  │ VB    │ ROOT         │
├────────────┼───────────┼───────┼───────┼──────────────┤
│ data       │ data      │ NOUN  │ NN    │ compound     │
├────────────┼───────────┼───────┼───────┼──────────────┤
│ elements   │ element   │ NOUN  │ NNS   │ dobj         │
├────────────┼───────────┼───────┼───────┼──────────────┤
│ surrounded │ surround  │ VERB  │ VBN   │ acl          │
├────────────┼───────────┼───────┼───────┼──────────────┤
│ quotation  │

# Transformers Setup

In [17]:
!pip install accelerate transformers
!pip install -U bitsandbytes
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# Import Llama 3.1

In [18]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [19]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"

In [20]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

## Quantization Configuration

In [21]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## Loading Tokenizer

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_name, token = HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [23]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    token=HF_TOKEN)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

## Generating Prompt

In [52]:
def format_for_transformer(results):
    prompt = f"""
    Role: {results['role']}.
    Action: {results['main_action']}.
    Conditions: {results['conditions']}.
    Original Text: {results['original_text']}.
    Key Dependencies: {', '.join([f'{dep[0]} ({dep[1]})' for dep in results['dependencies']])}.
    Generate a detailed Agile test case for this action.

    Return the output in this format:
    - Test Case Name: <Test Case Title>
    - Input Data: <Required input for the test>
    - Conditions: <Preconditions before execution>
    - Steps to Execute:
      1. <Step 1>
      2. <Step 2>
      3. <Step 3>
      4. ...
    - Expected Result: <What should happen if the test is successful>

    Do not change the structure or add extra details. Follow the exact format.
    """
    return prompt.strip()

In [53]:
def generate_test_case(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    device = next(model.parameters()).device  # Get model's device
    inputs = inputs.to(device)

    # Generate output
    output_tokens = model.generate(**inputs, max_new_tokens=256)

    # Decode and return the generated test case
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [54]:
test_cases = []
for i in range(5):
    prompt = format_for_transformer(res[i])
    test_case = generate_test_case(prompt)
    test_cases.append(test_case)
    print(test_case,"\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Role: Unknown.
    Action: processed.
    Conditions: Unknown.
    Original Text: ﻿As a Data user, I want to have the 12-19-2017 deletions processed.
.
    Key Dependencies: ﻿As (nummod), a (det), Data (compound), user (dep),, (punct), I (nsubj), want (dep), to (aux), have (xcomp), the (det), 12 (nummod), - (punct), 19 (nummod), - (punct), 2017 (prep), deletions (dobj),. (punct), 
 (dep).
    Generate a detailed Agile test case for this action.
    
    Return the output in this format:
    - Test Case Name: <Test Case Title>
    - Input Data: <Required input for the test>
    - Conditions: <Preconditions before execution>
    - Steps to Execute:
      1. <Step 1>
      2. <Step 2>
      3. <Step 3>
      4....
    - Expected Result: <What should happen if the test is successful>

    Do not change the structure or add extra details. Follow the exact format. """

- Test Case Name: Process 12-19-2017 Deletions
- Input Data: The system receives a request to process deletions from 12-19-2

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Role: UI designer.
    Action: redesign.
    Conditions: so that it matches the new Broker design styles.
    Original Text: As a UI designer, I want to redesign the Resources page, so that it matches the new Broker design styles.
.
    Key Dependencies: As (prep), a (det), UI (compound), designer (pobj),, (punct), I (nsubj), want (dep), to (aux), the (det), Resources (compound), page (dobj),, (punct), so (mark), that (mark), it (nsubj), matches (advcl), the (det), new (amod), Broker (compound), design (compound), styles (dobj),. (punct), 
 (dep).
    Generate a detailed Agile test case for this action.
    
    Return the output in this format:
    - Test Case Name: <Test Case Title>
    - Input Data: <Required input for the test>
    - Conditions: <Preconditions before execution>
    - Steps to Execute:
      1. <Step 1>
      2. <Step 2>
      3. <Step 3>
      4....
    - Expected Result: <What should happen if the test is successful>

    Do not change the structure or add extra d

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Role: UI designer.
    Action: report.
    Conditions: so that they are aware of their contributions to making Broker a better UX.
    Original Text: As a UI designer, I want to report to the Agencies about user testing, so that they are aware of their contributions to making Broker a better UX.
.
    Key Dependencies: As (prep), a (det), UI (compound), designer (pobj),, (punct), I (nsubj), want (dep), to (aux), to (prep), the (det), Agencies (pobj), about (prep), user (compound), testing (pobj),, (punct), so (mark), that (mark), they (nsubj), are (advcl), aware (acomp), of (prep), their (poss), contributions (pobj), to (prep), making (pcomp), Broker (nsubj), a (det), better (amod), UX (ccomp),. (punct), 
 (dep).
    Generate a detailed Agile test case for this action.
    
    Return the output in this format:
    - Test Case Name: <Test Case Title>
    - Input Data: <Required input for the test>
    - Conditions: <Preconditions before execution>
    - Steps to Execute:
      1. <Step

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Role: UI designer.
    Action: round.
    Conditions: so that I can get approvals from leadership.
    Original Text: As a UI designer, I want to move on to round 2 of DABS or FABS landing page edits, so that I can get approvals from leadership.
.
    Key Dependencies: As (prep), a (det), UI (compound), designer (pobj),, (punct), I (nsubj), want (dep), to (aux), move (xcomp), on (prt), to (aux), 2 (dobj), of (prep), DABS (nmod), or (cc), FABS (compound), landing (compound), page (conj), edits (pobj),, (punct), so (mark), that (mark), I (nsubj), can (aux), get (advcl), approvals (dobj), from (prep), leadership (pobj),. (punct), 
 (dep).
    Generate a detailed Agile test case for this action.
    
    Return the output in this format:
    - Test Case Name: <Test Case Title>
    - Input Data: <Required input for the test>
    - Conditions: <Preconditions before execution>
    - Steps to Execute:
      1. <Step 1>
      2. <Step 2>
      3. <Step 3>
      4....
    - Expected Result: <Wha

# Post Processing Helper Functions

In [122]:
import re
from tabulate import tabulate

def parse_test_case(raw_text):
    """Extracts structured fields from the test case output, handling different formats of Expected Output and multi-line steps."""

    fields = ["Test Case Name", "Input Data", "Conditions", "Steps to Execute", "Expected Result"]
    extracted_data = {field: "N/A" for field in fields}  # Default values

    for field in fields:
        if field == "Steps to Execute":
            # Extract multi-line steps (everything until "Expected Output" or "- Expected Output")
            match = re.search(r"Steps to Execute:\s*(.+?)\n\s*(?:- |)Expected Output:", raw_text, re.DOTALL)
        elif field == "Expected Result":
            # Extract everything after "Expected Output" or "- Expected Output"
            match = re.search(r"(?:- |)Expected Output:\s*(.+)", raw_text, re.DOTALL)
        else:
            # Extract single-line fields
            match = re.search(fr"{field}:\s*(.+)", raw_text)

        if match:
            extracted_data[field] = match.group(1).strip()

    return extracted_data

In [129]:
def parse_test_case(raw_text):
    """Extracts structured fields from the test case output, handling multi-line steps and expected results."""

    fields = ["Test Case ID","Test Case Name", "Input Data", "Conditions", "Steps to Execute", "Expected Result"]
    extracted_data = {field: "N/A" for field in fields}  # Default values

    for field in fields:
        if field == "Steps to Execute" or field == "- Steps to Execute:":
            # Extract steps (multi-line until "Expected Result:")
            match = re.search(r"Steps to Execute:\s*(.+?)\n\s*(- )?Expected Result:", raw_text, re.DOTALL)
        elif field == "Expected Result" or field == "- Expected Result:":
            # Extract everything after "Expected Result:"
            match = re.search(r"Expected Result:\s*(.+)", raw_text, re.DOTALL)
        else:
            # Extract normal fields (single line)
            match = re.search(fr"{field}:\s*(.+)", raw_text)

        if match:
            extracted_data[field] = match.group(1).strip()

    return extracted_data


In [130]:
def format_test_case(num, parsed_data):
    """Formats the extracted test case into a tabular format."""
    # Format "Steps to Execute" as multiple lines
    parsed_data["Test Case ID"] = f"TC-{num:03d}"

    parsed_data["Steps to Execute"] = parsed_data["Steps to Execute"]

    expected_result_sentences = re.split(r'(?<=\.)\s+', parsed_data["Expected Result"])  # Splitting at sentence end
    parsed_data["Expected Result"] = "\n".join(expected_result_sentences)  # Join each sentence with a new line

    table = [[field, parsed_data[field]] for field in parsed_data]
    return tabulate(table, headers=["Field", "Details"], tablefmt="grid")

# Post Processing Output

In [132]:
for i,test_case in enumerate(test_cases):
  normalized_output = clean_transformer_output(test_case)
  parsed_test_case = parse_test_case(normalized_output)
  print(format_test_case(i, parsed_test_case),"\n")

+------------------+--------------------------------------------------------------------------------------------------+
| Field            | Details                                                                                          |
+==================+==================================================================================================+
| Test Case ID     | TC-000                                                                                           |
+------------------+--------------------------------------------------------------------------------------------------+
| Test Case Name   | Process 12-19-2017 Deletions                                                                     |
+------------------+--------------------------------------------------------------------------------------------------+
| Input Data       | The system receives a request to process deletions from 12-19-2017.                              |
+------------------+--------------------